## Importing the required libraries 

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score

In [2]:
import warnings
warnings.simplefilter('ignore')

## Loading the dataset

In [3]:
data = pd.read_csv('dataset.csv')

In [4]:
data.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [5]:
data.shape

(22000, 2)

In [6]:
data['language'].unique()

array(['Estonian', 'Swedish', 'Thai', 'Tamil', 'Dutch', 'Japanese',
       'Turkish', 'Latin', 'Urdu', 'Indonesian', 'Portugese', 'French',
       'Chinese', 'Korean', 'Hindi', 'Spanish', 'Pushto', 'Persian',
       'Romanian', 'Russian', 'English', 'Arabic'], dtype=object)

In [7]:
data['language'].unique().size

22

**The dataset consists of 22000 records. It has 2 columns- 'Text' and 'language' corresponding to the text. There are 22 unique languages present in the dataset.**

## Checking for null and duplicate values 

In [8]:
# dropping duplicates (if any)
print('Before dropping duplicates: ', data.shape)
data.drop_duplicates()
print('After dropping duplicates: ', data.shape)

Before dropping duplicates:  (22000, 2)
After dropping duplicates:  (22000, 2)


**No duplicate records were present.**

In [9]:
# checking for NA values
data.isnull().sum()

Text        0
language    0
dtype: int64

**No NA values were present.**

## Example

In [10]:
data['Text'][3]

'விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திரிகை-விசாகப்பட்டின ஆசிரியர் சம்பத்துடன் இணைந்து விரிவுபடுத்தினார்  ஆண்டுகள் தொடர்ந்து செயலராக இருந்து தமிழ்மன்றத்தை நடத்திச் சென்றார்  கோவை செம்மொழி மாநாட்டில் "தமிழ்ச்சங்கங்களும் தமிழும்" எனும் தலைப்பில் பிற மாநிலங்களில் தமிழ்வளர்ச்சி பற்றி கட்டுரை வாசித்தார்'

In [11]:
data['language'][3]

'Tamil'

In [12]:
data['language'].value_counts()

Estonian      1000
Swedish       1000
English       1000
Russian       1000
Romanian      1000
Persian       1000
Pushto        1000
Spanish       1000
Hindi         1000
Korean        1000
Chinese       1000
French        1000
Portugese     1000
Indonesian    1000
Urdu          1000
Latin         1000
Turkish       1000
Japanese      1000
Dutch         1000
Tamil         1000
Thai          1000
Arabic        1000
Name: language, dtype: int64

**All languages have equal number of records in the dataset.**

Here, 'language' depends on the 'Text' provided. Hence, 'langauge' is the dependent and 'Text' is the independent variable. 

In [13]:
X = data['Text']
y = data['language']

In [14]:
X

0        klement gottwaldi surnukeha palsameeriti ning ...
1        sebes joseph pereira thomas  på eng the jesuit...
2        ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...
3        விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...
4        de spons behoort tot het geslacht haliclona en...
                               ...                        
21995    hors du terrain les années  et  sont des année...
21996    ใน พศ  หลักจากที่เสด็จประพาสแหลมมลายู ชวา อินเ...
21997    con motivo de la celebración del septuagésimoq...
21998    年月，當時還只有歲的她在美國出道，以mai-k名義推出首張英文《baby i like》，由...
21999     aprilie sonda spațială messenger a nasa și-a ...
Name: Text, Length: 22000, dtype: object

In [15]:
y

0        Estonian
1         Swedish
2            Thai
3           Tamil
4           Dutch
           ...   
21995      French
21996        Thai
21997     Spanish
21998     Chinese
21999    Romanian
Name: language, Length: 22000, dtype: object

ML algorithms only work with numberical data, so we have to first convert this text into labelled numerical data.

In [16]:
le = LabelEncoder()
encoded_lang = le.fit_transform(y)

In [17]:
# making a new column for encoded data
data['encoded_lang'] = encoded_lang

In [18]:
data.head()

,Text,language,encoded_lang
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian,4
1,sebes joseph pereira thomas på eng the jesuit...,Swedish,17
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai,19
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil,18
4,de spons behoort tot het geslacht haliclona en...,Dutch,2


There are 22 unique languages. So, each language is given a unique number between 0 to 21.  

Converting the text to lowercase.

In [19]:
# for storing new text
new_text = []

for text in X:
    text = text.lower()
    new_text.append(text)

In [20]:
new_text[2]

'ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เริ่มตั้งแต่ถนนสนามไชยถึงแม่น้ำเจ้าพระยาที่ถนนตก กรุงเทพมหานคร เป็นถนนรุ่นแรกที่ใช้เทคนิคการสร้างแบบตะวันตก ปัจจุบันผ่านพื้นที่เขตพระนคร เขตป้อมปราบศัตรูพ่าย เขตสัมพันธวงศ์ เขตบางรัก เขตสาทร และเขตบางคอแหลม'

**Getting the features out of the text.**

In [21]:
cv = CountVectorizer(max_features=30000)
X = cv.fit_transform(new_text).toarray()

In [22]:
X.shape

(22000, 30000)

22000 rows and 30000 columns/features.

**Splitting the dataset into training and testing data.**

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Multinomial NB 

In [24]:
mnb = MultinomialNB()

In [25]:
mnb.fit(X_train, y_train)

MultinomialNB()

In [26]:
y_pred = mnb.predict(X_test)

**Accuracy**

In [27]:
ac_mnb = accuracy_score(y_test, y_pred)
print(ac_mnb)

0.9340909090909091


## Bernoulli NB

In [31]:
model2 = BernoulliNB()

In [32]:
model2.fit(X_train, y_train)

BernoulliNB()

In [33]:
y_pred2 = model2.predict(X_test)

In [34]:
y_pred2

array(['Latin', 'Romanian', 'Turkish', ..., 'Thai', 'Urdu', 'Swedish'],
      dtype='<U10')

In [35]:
ac2 = accuracy_score(y_test, y_pred2)
print(ac2)

0.9279545454545455
